https://www.transtats.bts.gov/Fields.asp?Table_ID=236

In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 10 # Ensure reproducible results
%matplotlib inline
plt.style.use('ggplot')
from flights_delay.feature_processing import FlightDelayFeatureProcessing
from helpers.outliers import MyOutliers
from helpers.feature_engineering import makeBinaryClassification, getUniqueValuesPerFeature
from sklearn.model_selection import train_test_split

In [4]:
data_relative_path = '../Data'

# Flying to New York City - Raw Data Preprocess

In [5]:
path_data = os.path.realpath(os.path.join(os.getcwd(), data_relative_path, 'raw_data.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/raw_data.csv'

In [6]:
df = pd.read_csv(path_data, delimiter = ',', index_col=0, header=0)
df.shape

(492181, 39)

In [7]:
print("There are {} entries and {} columns in the DataFrame"\
      .format(df.shape[0], df.shape[1]))

There are 492181 entries and 39 columns in the DataFrame


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492181 entries, 0 to 492180
Data columns (total 39 columns):
YEAR                   492181 non-null int64
QUARTER                492181 non-null int64
MONTH                  492181 non-null int64
DAY_OF_MONTH           492181 non-null int64
DAY_OF_WEEK            492181 non-null int64
FL_DATE                492181 non-null object
UNIQUE_CARRIER         492181 non-null object
AIRLINE_ID             492181 non-null int64
CARRIER                492181 non-null object
FL_NUM                 492181 non-null int64
ORIGIN_AIRPORT_ID      492181 non-null int64
ORIGIN                 492181 non-null object
ORIGIN_CITY_NAME       492181 non-null object
ORIGIN_STATE_ABR       492181 non-null object
ORIGIN_STATE_NM        492181 non-null object
DEST_AIRPORT_ID        492181 non-null int64
DEST                   492181 non-null object
DEST_CITY_NAME         492181 non-null object
DEST_STATE_ABR         492181 non-null object
DEST_STATE_NM          4

In [9]:
df_0 = FlightDelayFeatureProcessing.dropFlightNumber(df)
df_0.shape

(492181, 38)

#### Create is flight delayed boolean

In [10]:
df_1 = FlightDelayFeatureProcessing.createIsDelayedCol(df_0, shuffle_it=False)
df_1.shape

(492181, 39)

In [11]:
np.all( df_1['ARR_DEL15'] == df_1['IS_DELAYED'] )

False

#### Remove all arrival columns except arrival delay group (useful for narrow the dataset for binary classification)

In [12]:
#df_2 = FlightDelayFeatureProcessing.removeArrivalAttrsExceptArrDelay(df_1)
#df_2 = FlightDelayFeatureProcessing.removeArrivalAttrs(df_1)
df_2 = FlightDelayFeatureProcessing.removeArrivalAttrsExceptArrDelayGroup(df_1)
df_2.shape

(492181, 34)

#### Remove also elapsed time because it indicates when the airplane arrives, which is what we want to predict

In [13]:
df_3 = FlightDelayFeatureProcessing.dropActualElapsedTime(df_2)
df_3.shape

(492181, 33)

In [14]:
df_4 = FlightDelayFeatureProcessing.eraseCancelledFlights(df_3)
df_4.shape

(483232, 33)

In [15]:
df_5 = FlightDelayFeatureProcessing.removeCancelledAndFlights(df_4)
df_5.shape

(483232, 31)

#### Erase rows with nulls

In [16]:
df_6 = FlightDelayFeatureProcessing.eraseRowsWithNulls(df_5)
df_6.shape

(481662, 31)

In [17]:
FlightDelayFeatureProcessing.getAllRowsWithAtLeastOneNull(df_6)

set()

### CONCLUSION: ONLY CANCELLED FLIGHTS HAD MISSING DATA, ALL THE REST ARE OK

#### CRS departure time is not a useful information

In [18]:
df_7 = FlightDelayFeatureProcessing.removeCRSDeptTime(df_6)
df_7.shape

(481662, 30)

#### Remove redundant State names

In [19]:
df_8 = FlightDelayFeatureProcessing.removeRedundantStateNames(df_7)
df_8.shape

(481662, 28)

#### Remove Origin Airport Id, Dest Airport Id and Airline Id

In [20]:
df_9 = FlightDelayFeatureProcessing.removeIdColsAlreadyRepresentedByStrCols(df_8)
df_9.shape

(481662, 25)

#### remove carrier because it is already represented in unique carrier

In [21]:
df_10 = FlightDelayFeatureProcessing.removeCarrier(df_9)
df_10.shape

(481662, 24)

#### remove year since we are dealing with only 2016 and also remove flight date

In [22]:
df_11 = FlightDelayFeatureProcessing.dropYearAndDate(df_10)
df_11.shape

(481662, 22)

In [23]:
df_11.dtypes

QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
UNIQUE_CARRIER       object
ORIGIN               object
ORIGIN_CITY_NAME     object
ORIGIN_STATE_ABR     object
DEST                 object
DEST_CITY_NAME       object
DEST_STATE_ABR       object
DEP_TIME            float64
DEP_DELAY           float64
DEP_DELAY_NEW       float64
DEP_DEL15           float64
DEP_DELAY_GROUP     float64
DEP_TIME_BLK         object
ARR_DELAY_GROUP     float64
CRS_ELAPSED_TIME    float64
DISTANCE            float64
DISTANCE_GROUP        int64
IS_DELAYED             bool
dtype: object

## Too many instances for our hardware, let's reduce them by taking the some kind of popular destinations and kind of popular origins - kind of popular trips

In [24]:
testdf = df_11.copy()

In [25]:
countPerOrigin = testdf.groupby(['ORIGIN', 'DEST']).size()

In [26]:
len(testdf[np.logical_and(testdf['ORIGIN'] == 'BWI',  testdf['DEST'] == 'ALB')])

1885

In [27]:
countSorted = countPerOrigin.sort_values()[::-1]
countSorted

ORIGIN  DEST
JFK     LAX     12445
LAX     JFK     12435
LGA     ORD      9803
ORD     LGA      9680
SFO     JFK      7997
LGA     ATL      7992
ATL     LGA      7977
JFK     SFO      7976
LGA     BOS      6354
MIA     LGA      6335
BOS     LGA      6329
LGA     MIA      6311
        FLL      5543
FLL     LGA      5538
JFK     BOS      5418
BOS     JFK      5411
JFK     MCO      5392
MCO     JFK      5375
DFW     LGA      4417
LGA     DFW      4415
JFK     FLL      4359
FLL     JFK      4334
LGA     MCO      4316
JFK     SJU      4288
MCO     LGA      4283
SJU     JFK      4267
LGA     CLT      4067
JFK     LAS      4066
LAS     JFK      4060
CLT     LGA      4007
                ...  
MHT     LGA        10
JFK     JAC        10
ALB     DEN         9
DEN     ALB         9
        BUF         9
BUF     DEN         9
BZN     LGA         8
LGA     BZN         8
BUF     PHL         8
PHL     BUF         8
CVG     JFK         7
JFK     CVG         6
        PHL         5
LGA     MEM        

In [28]:
mostPopular = countSorted[:10].copy()
mostPopular

ORIGIN  DEST
JFK     LAX     12445
LAX     JFK     12435
LGA     ORD      9803
ORD     LGA      9680
SFO     JFK      7997
LGA     ATL      7992
ATL     LGA      7977
JFK     SFO      7976
LGA     BOS      6354
MIA     LGA      6335
dtype: int64

In [29]:
popular = pd.DataFrame()

In [30]:
for key in mostPopular.keys():
    print key
    subset = testdf[np.logical_and(testdf['ORIGIN'] == key[0], testdf['DEST'] == key[1])]
    print len(subset)
    print
    popular = pd.concat((
            popular, subset            
        ))

('JFK', 'LAX')
12445

('LAX', 'JFK')
12435

('LGA', 'ORD')
9803

('ORD', 'LGA')
9680

('SFO', 'JFK')
7997

('LGA', 'ATL')
7992

('ATL', 'LGA')
7977

('JFK', 'SFO')
7976

('LGA', 'BOS')
6354

('MIA', 'LGA')
6335



In [31]:
np.sum(mostPopular)

88994

In [32]:
len(popular)

88994

In [33]:
len(popular[popular['IS_DELAYED']])

19449

In [34]:
# fig, ax = plt.subplots(figsize=(16,5))

# sns.barplot(x='ORIGIN', y=countPerOrigin.values, data=df_11, #ax=ax,
#             color = "r")

# plt.show()

### Remove Outliers

In [35]:
popular.dtypes

QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
UNIQUE_CARRIER       object
ORIGIN               object
ORIGIN_CITY_NAME     object
ORIGIN_STATE_ABR     object
DEST                 object
DEST_CITY_NAME       object
DEST_STATE_ABR       object
DEP_TIME            float64
DEP_DELAY           float64
DEP_DELAY_NEW       float64
DEP_DEL15           float64
DEP_DELAY_GROUP     float64
DEP_TIME_BLK         object
ARR_DELAY_GROUP     float64
CRS_ELAPSED_TIME    float64
DISTANCE            float64
DISTANCE_GROUP        int64
IS_DELAYED             bool
dtype: object

From all the features only the dep delay, distance, and arrival delay make sense to be considered for outliers

In [36]:
outlier_columns = ['DEP_DELAY', 'DISTANCE']#, 'ARR_DELAY']

In [37]:
types = popular[outlier_columns].dtypes
for col in outlier_columns:
    types[col] = MyOutliers().getLooseBoundaries(popular[col], k=3)
bounds_k3 = types.copy()

bounds_k3

DEP_DELAY    (-107.639863362, 132.353776659)
DISTANCE                   (-4493.0, 7701.0)
dtype: object

In [38]:
outliersCount = MyOutliers.countOutliersDataPoints(popular, bounds_k3)
outliersCount

DEP_DELAY    2123
DISTANCE        0
dtype: object

In [39]:
100 * np.sum(outliersCount) / len(popular)

2.385554082297683

In [40]:
inds = MyOutliers().getOutlierDataPoints(popular, bounds_k3)

#### ARE ALL INDICES delayed flights ?

In [41]:
outliers = popular.iloc[list(inds)]

In [42]:
len(outliers[outliers['IS_DELAYED']]) / len(outliers)

1.0

In [43]:
survivors, survivor_inds = MyOutliers().removeOutliers(popular, bounds_k3)
survivors.shape

(86871, 22)

In [44]:
survivors.dtypes

QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
UNIQUE_CARRIER       object
ORIGIN               object
ORIGIN_CITY_NAME     object
ORIGIN_STATE_ABR     object
DEST                 object
DEST_CITY_NAME       object
DEST_STATE_ABR       object
DEP_TIME            float64
DEP_DELAY           float64
DEP_DELAY_NEW       float64
DEP_DEL15           float64
DEP_DELAY_GROUP     float64
DEP_TIME_BLK         object
ARR_DELAY_GROUP     float64
CRS_ELAPSED_TIME    float64
DISTANCE            float64
DISTANCE_GROUP        int64
IS_DELAYED             bool
dtype: object

## Make it a balanced classification problem

In [45]:
binary_df = makeBinaryClassification(survivors, survivors['ARR_DELAY_GROUP'] <= 0)
binary_df.shape

(34652, 22)

In [46]:
assert len(binary_df[binary_df['IS_DELAYED']]) * 2  == len(binary_df)

In [47]:
final_df = FlightDelayFeatureProcessing.removeArrDelayGroup(binary_df)
final_df.shape

(34652, 21)

### Unique values per feature

In [48]:
getUniqueValuesPerFeature(final_df)


QUARTER: 
[2 1 3 4]

MONTH: 
[ 5  2  8  7 12  3  6 10  9  1  4 11]

DAY_OF_MONTH: 
[ 9  5 30  8 14 22  2  3 12 11 16 29 21 31 25 13  6 24 18 28  1 20 23 27 15
 17 26  4 19  7 10]

DAY_OF_WEEK: 
[1 5 2 3 4 6 7]

UNIQUE_CARRIER: 
['AA' 'VX' 'DL' 'WN' 'B6' 'UA' 'NK' 'OO' 'F9']

ORIGIN: 
['LGA' 'LAX' 'JFK' 'ATL' 'SFO' 'ORD' 'MIA']

ORIGIN_CITY_NAME: 
['New York, NY' 'Los Angeles, CA' 'Atlanta, GA' 'San Francisco, CA'
 'Chicago, IL' 'Miami, FL']

ORIGIN_STATE_ABR: 
['NY' 'CA' 'GA' 'IL' 'FL']

DEST: 
['BOS' 'JFK' 'SFO' 'ATL' 'LGA' 'ORD' 'LAX']

DEST_CITY_NAME: 
['Boston, MA' 'New York, NY' 'San Francisco, CA' 'Atlanta, GA'
 'Chicago, IL' 'Los Angeles, CA']

DEST_STATE_ABR: 
['MA' 'NY' 'CA' 'GA' 'IL']

DEP_TIME: 
Too large to show

DEP_DELAY: 
Too large to show

DEP_DELAY_NEW: 
[   0.    2.   45.   46.   14.   72.   57.   78.  122.   16.   55.    1.
   11.   33.  107.   20.   62.   49.   40.   32.   93.    6.   15.   76.
  117.   47.    7.   69.   10.   91.   42.   67.   92.   85.   29.   17

### Save Train and Test Data

In [49]:
data_train, data_test = train_test_split(final_df, test_size=0.1, random_state=random_state)
data_train.shape, data_test.shape

((31186, 21), (3466, 21))

In [50]:
# data_train.to_csv(
#     os.path.realpath(os.path.join(os.getcwd(), '../Data', 'train_data_v2.csv')),
#     index=False)

In [51]:
# data_test.to_csv(
#     os.path.realpath(os.path.join(os.getcwd(), '../Data', 'test_data_v2.csv')),
#     index=False)